In [2]:
import pandas as pd
import numpy as np
import json

# 시각화 라이브러리
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
from folium.plugins import MarkerCluster

In [3]:
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

In [4]:
file_path = './data/'